In [1]:
import pandas as pd
import numpy as np
from catboost import CatBoostClassifier, Pool

In [2]:
from prep_func import prep, get_res_from_df, read_data

In [3]:
from config import train_datapath, test_datapath, not_fit_cols, my_params, drops, model_path

In [4]:
df_train, df_test = read_data(train_datapath, test_datapath, not_fit_cols)

In [5]:
%%time
full_train_poly = prep(df_train[['id', 'crop', '.geo']])
full_test_poly = prep(df_test[['id', '.geo']])

CPU times: user 28.6 s, sys: 244 ms, total: 28.9 s
Wall time: 28.9 s


In [7]:
res_train = get_res_from_df(full_train_poly, full_train_poly)
res_test = get_res_from_df(full_test_poly, full_train_poly)

  0%|          | 0/4898 [00:00<?, ?it/s]

  0%|          | 0/2071 [00:00<?, ?it/s]

In [9]:
t = res_train.merge(df_train.merge(
    full_train_poly[['id', 'lat', 'lon']], on='id', how='left'), how='right', on ='id')
t_test = res_test.merge(df_test.merge(
    full_test_poly[['id', 'lat', 'lon']], on='id', how='left'), how='right', on ='id')

In [10]:
sols = {}
for i in range(7):
    sols[i] = []

In [11]:
for i, params in enumerate(my_params):
    cb = CatBoostClassifier(**params, n_estimators=1286, 
                            max_depth = 6, random_state=123)
    all_pool = Pool(t.drop(list(drops)+list(['crop', 'id', '.geo']), axis=1), t['crop'])
    cb.fit(all_pool,  verbose=500, early_stopping_rounds=50)
    cb.save_model(f'{model_path}/model_{i}')
    for i in cb.classes_:
        sols[i].append(cb.predict_proba(t_test[cb.feature_names_])[:, i])

0:	learn: 1.7675078	total: 77.4ms	remaining: 1m 39s
500:	learn: 0.0377853	total: 9.89s	remaining: 15.5s
1000:	learn: 0.0110858	total: 19.6s	remaining: 5.58s
1285:	learn: 0.0070111	total: 25.2s	remaining: 0us
0:	learn: 1.5338498	total: 35.9ms	remaining: 46.1s
500:	learn: 0.0116127	total: 12.1s	remaining: 18.9s
1000:	learn: 0.0041001	total: 24.1s	remaining: 6.86s
1285:	learn: 0.0029203	total: 31s	remaining: 0us
0:	learn: 1.6362025	total: 31ms	remaining: 39.8s
500:	learn: 0.0297294	total: 10.4s	remaining: 16.2s
1000:	learn: 0.0107092	total: 20.7s	remaining: 5.89s
1285:	learn: 0.0075856	total: 26.5s	remaining: 0us


In [12]:
for i in cb.classes_:
    t_test[i] = np.min(sols[i], axis=0)
t_test['crop'] = t_test[cb.classes_].idxmax(axis=1)
t_test = t_test.merge(t[['.geo', 'crop']], on = '.geo', how='left')

In [14]:
t_test['crop'] = 1
t_test.loc[t_test.crop_y.isnull(), 'crop'] = t_test.loc[t_test.crop_y.isnull(), 'crop_x'].astype(int)
t_test.loc[~t_test.crop_y.isnull(), 'crop'] = t_test.loc[~t_test.crop_y.isnull(), 'crop_y'].astype(int)

In [15]:
t_test[['id', 'crop']].to_csv('solution.csv', index=False)